# Uploading building data shapefiles from Open Street Maps

### Building data to be uploaded if wishing to base the road-hospital connection/integration based on proximity to polygon instead of point type data. This is not necessary but can be useful for additional spatial queries where you can identify which buildings are hospitals.

In [1]:
'''
Loading imports needed to call all libraries to perform uploading of datasets related to national transit
Please make sure to install requirements as per README.md file's instructions.
'''
import geopandas as gpd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import numpy as np

# Connect to engine after creation

In [2]:
'''
Load environment variables in a safe way by storing them on your personal .env file
and ensuring said file is listed within your .gitignore file.

'''
load_dotenv()

USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
SERVER = os.getenv("SERVER")
DATABASE = os.getenv("DATABASE")
PORT = '5432'

db_connection_url = f"postgresql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}"
engine = create_engine(db_connection_url)

In [3]:
'''
Including geoJSON files from Open Street Maps
from https://download.geofabrik.de/north-america/us/georgia-latest-free.shp.zip
Please note, the shapefiles selected within the zip file are only the roads shapefiles and all other layers are not needed.

I recommend saving said files within a folder called georgia_osm which will contain the following:

- gis_osm_buildings_a_free_1.cpg
- gis_osm_buildings_a_free_1.dbf
- gis_osm_buildings_a_free_1.prj
- gis_osm_buildings_a_free_1.shp
- gis_osm_buildings_a_free_1.shx

'''

# read in the data
PATH = os.getenv("BUILDINGS")
gdf = gpd.read_file(PATH)

In [4]:
gdf # Check if geodataframe was created successfully

,osm_id,code,fclass,name,type,geometry
0,9047651,1500,building,NaN,NaN,"POLYGON ((-82.01972 33.09772, -82.01957 33.097..."
1,9202141,1500,building,NaN,NaN,"POLYGON ((-82.86844 34.11479, -82.86811 34.115..."
2,9518929,1500,building,NaN,NaN,"POLYGON ((-82.35683 31.22072, -82.35669 31.220..."
3,24232553,1500,building,Hurt Building,commercial,"POLYGON ((-84.38778 33.75436, -84.38778 33.754..."
4,24445165,1500,building,Concourse A,NaN,"POLYGON ((-84.43979 33.64077, -84.43979 33.640..."
...,...,...,...,...,...,...
2493388,1159481198,1500,building,NaN,house,"POLYGON ((-84.23344 34.77340, -84.23331 34.773..."
2493389,1159481199,1500,building,NaN,house,"POLYGON ((-84.23456 34.77342, -84.23440 34.773..."
2493390,1159481200,1500,building,NaN,house,"POLYGON ((-84.23512 34.77336, -84.23498 34.773..."
2493391,1159481201,1500,building,NaN,house,"POLYGON ((-84.23627 34.77273, -84.23624 34.772..."


In [ ]:
# Push the geodataframe to postgresql database
gdf.to_postgis("ga_buildings", engine, index=False, if_exists='replace')

# Uploading ga_boundary data for additional spatial query capabilitites

### Data for GA boundary comes from geojson file downloaded at https://hub.arcgis.com/datasets/GARC::georgia-state-boundary/explore?location=32.645735%2C-83.120000%2C7.92

In [3]:
# Retrieve path and access file
PATH = os.getenv("GA_BOUNDARY")
gdf = gpd.read_file(PATH)

In [4]:
# Check if successful dataframe creation
gdf

,OBJECTID,Name,GlobalID,last_edited_date,geometry
0,1,Georgia,{626E21AE-9939-45DF-A024-6A269E0C1934},NaN,"POLYGON ((-83.10854 35.00078, -83.12076 35.000..."


In [5]:
# Push the geodataframe to postgresql database
gdf.to_postgis("ga_boundary", engine, index=False, if_exists='replace')